In [ ]:
# 5.2.8 Practical activity: Search the Twitter API

#Scenario

#Earlier, as a data analyst at a financial institution, your manager tasked you with investigating Bitcoin in terms of 
#future growth and its use in the United States. Previously, you accessed Bitcoin data through the Coingecko API. Now, 
#your manager asks you to turn your attention to Twitter.

#Your manager then wants you to cross-check trending topics between the United States (New York) and the UK (London), to 
#see what people are talking about in both countries and if Bitcoin forms part of the larger conversation. If Bitcoin is 
#not a shared trending topic, then she asks that you search Twitter for #Bitcoin and provide a JSON printout. 

#Approach

#To begin exploring Bitcoin on Twitter you will need to start a new Jupyter Notebook and connect to the Twitter API with 
#your OAuth authentication. Follow this code snippet to prepare your Jupyter Notebook with the Twitter.API.

# Copy the YAML file and your Twitter keys over before you start to work.
import yaml
from yaml.loader import SafeLoader
from twitter import *

# Import the YAML file - remember to specify the whole path.
twitter_creds = yaml.safe_load(open('twitter_tmp_updated.yaml', 'r').read())

# Pass your twitter credentials.
twitter_api = Twitter(auth=OAuth(twitter_creds['access_token'],
                                 twitter_creds['access_token_secret'], 
                                 twitter_creds['api_key'],
                                 twitter_creds['api_secret_key'] ))

# You will then need to: 

#Check your connection to the API. Confirm the connection with the #python search.

python_tweets_test=twitter_api.search.tweets(q="#python")
print(python_tweets_test)

#Search trends worldwide for the IDs of the required cities (US as New York and UK as London)
#    Identify the trends worldwide.

trends_worldwide = twitter_api.trends.available()
print(len(trends_worldwide))
trends_worldwide[0]

#    Identify New York and London with the woeid method.

list_of_names_London = [_ for _ in trends_worldwide if _["name"] == "London"]
print(len(list_of_names_London))
list_of_names_London[0]["woeid"]

#if uyou use list_of_names_London = [_["name"] for _ in trends_worldwide if _["name"] == "London"]
# it will only return the name column!!

#    Find a list of the trends specific to each city and save it as a Pandas DataFrame.
#    Limit the output to more than 50,000 tweets.
#    Save the output as a CSV file.

import pandas as pd
London_trends = twitter_api.trends.place(_id = list_of_names_London[0]['woeid'])
London_trends_df = pd.DataFrame(London_trends[0]["trends"])
London_trends_df_new = London_trends_df[London_trends_df["tweet_volume"]>50000]
London_trends_df_new
London_trends_df_new.to_csv("London_trends_df_over_50k_tweets.csv")

list_of_names_NY = [_ for _ in trends_worldwide if _["name"] == "New York"]
print(len(list_of_names_NY))
#list_of_names_NY[0]["woeid"]

import pandas as pd
NY_trends = twitter_api.trends.place(_id = list_of_names_NY[0]['woeid'])
NY_trends_df = pd.DataFrame(NY_trends[0]["trends"])
NY_trends_df_new = NY_trends_df[NY_trends_df["tweet_volume"]>50000]
NY_trends_df_new
NY_trends_df_new.to_csv("NY_trends_df_over_50k_tweets.csv")

#Find the common trends between New York and London.
    #Use the woeid of each city as an identifier.
    #Search the Twitter API for trends in both cities with the woeid.
    #Print the output with json.dumps() and indent of 4.
    
import json 
London_trends_list = [trend["name"] for trend in London_trends[0]["trends"]]
print(json.dumps(London_trends_list, indent=4))

NY_trends_list = [trend["name"] for trend in NY_trends[0]["trends"]]
print(json.dumps(NY_trends_list, indent=4))

    #Create a list of the common topics and use the set() function to determine the intersection.
    #Print the common trend.
    
London_trends_set = set(London_trends_list)
NY_trends_set = set(NY_trends_list)

common_trends = London_trends_set.intersection(NY_trends_set)
common_trends

#Search Twitter for #Bitcoin and print the output with and json.dumps() and indent of 4.

search_results = twitter_api.search.tweets(q="#bitcoin", Count=100)
statuses = search_results["statuses"]

for _ in range(5):
    print("Length of statuses", len(statuses))
    try:
        next_results = search_results['search_metadata']['next_results']
    except KeyError: # No more results when next_results doesn't exist
        break
        
    # Create a dictionary from next_results
    kwargs = dict([ kv.split('=') for kv in next_results[1:].split("&") ])
    
    search_results = twitter_api.search.tweets(**kwargs)
    statuses += search_results['statuses']

print(json.dumps(statuses[1], indent=4))

#Save the Notebook to your Github repository by uploading and committing any changes.

# I GIVE UP THIS PARTT

# LSE'S SOLUTOIN

### LSE Data Analytics Online Career Accelerator

# DA201: Data Analytics using Python

## Practical activity: Search the Twitter API

**Important**

Please take note that you will work with the Twitter API. Keep in mind that the Twitter API is based on live and current events. Therefore, your output will differ from the outputs provided. For example, the colour of apples was trending yesterday, but today the aerodynamics of aeroplanes is trending.

**This is the solution to the activity.**

The story of Bitcoin and other cryptocurrencies has captured investors like few financial stories have. Many finance firms are looking to invest in the crypto market. As a data analyst at a financial institution, your manager has tasked you with investigating Bitcoin in a little more detail, particularly in terms of future growth of the currency and its use in the United States. 

Earlier, as a data analyst at a financial institution, your manager tasked you with investigating Bitcoin in terms of future growth and its use in the United States. Previously, you accessed Bitcoin data through the Coingecko API. Now, your manager asks you to turn your attention to Twitter, particularly tweets on Bitcoin and cryptocurrency in general. Your manager particularly wants you to check if Bitcoin is trending in `New York, Los Angeles, Sydney, Auckland, and Dubai`.

She also wants to see a DataFrame of topics with over `200,000` tweets for each city. 

Your manager then wants you to cross-check trending topics between the `United States and the UK`, to see what people are talking about in both countries, and if Bitcoin forms part of the larger conversation. If Bitcoin is not a shared trending topic, then she asks that you search Twitter for `#Bitcoin` and two other cryptocurrency hashtags of your choice, and analyse the top two tweets you return for each hashtag, particularly in terms of their popularity.

## 1. Prepare your workstation

In [3]:
# Copy the YAML file and your Twitter keys over to this Jupyter Notebook before you start to work.
import yaml
from yaml.loader import SafeLoader
from twitter import *

# Import the YAML file - remember to specify the whole path.
twitter_creds = yaml.safe_load(open('twitter_tmp_updated.yaml', 'r').read())

# Pass your Twitter credentials.
twitter_api = Twitter(auth=OAuth(twitter_creds['access_token'],
                                 twitter_creds['access_token_secret'], 
                                 twitter_creds['api_key'],
                                 twitter_creds['api_secret_key'] ))

In [4]:
# See if you are connected.
print(twitter_api)

In [5]:
# Run a test with #python.
python_tweets = twitter_api.search.tweets(q="#python")

# View the output.
print(python_tweets)

{'statuses': [{'created_at': 'Mon Jul 04 05:53:10 +0000 2022', 'id': 1543835288341868544, 'id_str': '1543835288341868544', 'text': 'Education is something we have to keep pursuing day after day"\n"Choose a job you love, and you will never have to w… https://t.co/YOqTQDzjYy', 'truncated': True, 'entities': {'hashtags': [], 'symbols': [], 'user_mentions': [], 'urls': [{'url': 'https://t.co/YOqTQDzjYy', 'expanded_url': 'https://twitter.com/i/web/status/1543835288341868544', 'display_url': 'twitter.com/i/web/status/1…', 'indices': [117, 140]}]}, 'metadata': {'iso_language_code': 'en', 'result_type': 'recent'}, 'source': '<a href="https://mobile.twitter.com" rel="nofollow">Twitter Web App</a>', 'in_reply_to_status_id': None, 'in_reply_to_status_id_str': None, 'in_reply_to_user_id': None, 'in_reply_to_user_id_str': None, 'in_reply_to_screen_name': None, 'user': {'id': 1148120176023330816, 'id_str': '1148120176023330816', 'name': 'Pyspiders', 'screen_name': 'pyspiders', 'location': 'Bangalore

## 2. Identify New York and London

In [6]:
# Determine worldwide trends.
trends_worldwide = twitter_api.trends.available()

# How many trends are available?
print(len(trends_worldwide))

# Example of trends_worldwide.
trends_worldwide[0]

467


{'name': 'Worldwide',
 'placeType': {'code': 19, 'name': 'Supername'},
 'url': 'http://where.yahooapis.com/v1/place/1',
 'parentid': 0,
 'country': '',
 'woeid': 1,
 'countryCode': None}

## New York

In [7]:
# Find New York.
our_city = 'New York'

# Create a variable.
new_york = [_ for _ in trends_worldwide if _['name'] == our_city]

# View the output.
print(len(new_york))

# Use index to find New York.
new_york[0]

# List of where on earth identifier (woeid).
new_york[0]['woeid']

1


2459115

## London

In [8]:
# Find London.
our_city_2 = 'London'

# Create a variable.
london = [_ for _ in trends_worldwide if _['name'] == our_city_2]

# View the output.
print(len(london))

# Use index to find London.
london[0]

# List of where on earth identifier (woeid).
london[0]['woeid']

1


44418

## 3. Common trends

## New York

In [9]:
# Look at trends in New York.
new_york_trends = twitter_api.trends.place(_id = new_york[0]['woeid'])

# View the output.
new_york_trends

[{'trends': [{'name': '#July4th',
    'url': 'http://twitter.com/search?q=%23July4th',
    'promoted_content': None,
    'query': '%23July4th',
    'tweet_volume': 14983},
   {'name': '#JaylandWalker',
    'url': 'http://twitter.com/search?q=%23JaylandWalker',
    'promoted_content': None,
    'query': '%23JaylandWalker',
    'tweet_volume': 56690},
   {'name': 'Breaking Bad',
    'url': 'http://twitter.com/search?q=%22Breaking+Bad%22',
    'promoted_content': None,
    'query': '%22Breaking+Bad%22',
    'tweet_volume': 17501},
   {'name': 'New Edition',
    'url': 'http://twitter.com/search?q=%22New+Edition%22',
    'promoted_content': None,
    'query': '%22New+Edition%22',
    'tweet_volume': None},
   {'name': 'The Wire',
    'url': 'http://twitter.com/search?q=%22The+Wire%22',
    'promoted_content': None,
    'query': '%22The+Wire%22',
    'tweet_volume': 14731},
   {'name': 'Happy 4th of July',
    'url': 'http://twitter.com/search?q=%22Happy+4th+of+July%22',
    'promoted_conte

In [10]:
# Look at the output as a DataFrame.
# Import Pandas.
import pandas as pd

# Create a DataFrame.
new_york_trends_pd = pd.DataFrame(new_york_trends[0]['trends'])

# View a DataFrame.
new_york_trends_pd

,name,url,promoted_content,query,tweet_volume
0,#July4th,http://twitter.com/search?q=%23July4th,None,%23July4th,14983.0
1,#JaylandWalker,http://twitter.com/search?q=%23JaylandWalker,None,%23JaylandWalker,56690.0
2,Breaking Bad,http://twitter.com/search?q=%22Breaking+Bad%22,None,%22Breaking+Bad%22,17501.0
3,New Edition,http://twitter.com/search?q=%22New+Edition%22,None,%22New+Edition%22,NaN
4,The Wire,http://twitter.com/search?q=%22The+Wire%22,None,%22The+Wire%22,14731.0
5,Happy 4th of July,http://twitter.com/search?q=%22Happy+4th+of+Ju...,None,%22Happy+4th+of+July%22,44754.0
6,The Sopranos,http://twitter.com/search?q=%22The+Sopranos%22,None,%22The+Sopranos%22,NaN
7,#ESSENCEFest,http://twitter.com/search?q=%23ESSENCEFest,None,%23ESSENCEFest,54535.0
8,#USFLChampionship,http://twitter.com/search?q=%23USFLChampionship,None,%23USFLChampionship,NaN
9,The Good Place,http://twitter.com/search?q=%22The+Good+Place%22,None,%22The+Good+Place%22,11283.0


In [11]:
# Narrow list down to 50,000 tweets.
new_york_trends_over50k_pd = new_york_trends_pd[new_york_trends_pd['tweet_volume'] > 50000]\
.sort_values('tweet_volume', ascending=False)

# View the output.
print(new_york_trends_over50k_pd.shape)
new_york_trends_over50k_pd

(9, 5)


,name,url,promoted_content,query,tweet_volume
17,Chan,http://twitter.com/search?q=Chan,None,Chan,302131.0
26,Fireworks,http://twitter.com/search?q=Fireworks,None,Fireworks,189880.0
12,The Office,http://twitter.com/search?q=%22The+Office%22,None,%22The+Office%22,144882.0
49,The Americans,http://twitter.com/search?q=%22The+Americans%22,None,%22The+Americans%22,113009.0
14,Independence Day,http://twitter.com/search?q=%22Independence+Da...,None,%22Independence+Day%22,101543.0
39,Denmark,http://twitter.com/search?q=Denmark,None,Denmark,67607.0
1,#JaylandWalker,http://twitter.com/search?q=%23JaylandWalker,None,%23JaylandWalker,56690.0
7,#ESSENCEFest,http://twitter.com/search?q=%23ESSENCEFest,None,%23ESSENCEFest,54535.0
38,Pennsylvania,http://twitter.com/search?q=Pennsylvania,None,Pennsylvania,53331.0


In [12]:
# Save output as a CSV file.
new_york_trends_over50k_pd.to_csv('new_york_trends_over50k.csv', index=False)

## London

In [13]:
# Look at trends in London.
london = twitter_api.trends.place(_id = london[0]['woeid'])

# View the output.
london

[{'trends': [{'name': '#BritishGP',
    'url': 'http://twitter.com/search?q=%23BritishGP',
    'promoted_content': None,
    'query': '%23BritishGP',
    'tweet_volume': 553495},
   {'name': '#SilverstoneGP',
    'url': 'http://twitter.com/search?q=%23SilverstoneGP',
    'promoted_content': None,
    'query': '%23SilverstoneGP',
    'tweet_volume': 118576},
   {'name': 'Zhou',
    'url': 'http://twitter.com/search?q=Zhou',
    'promoted_content': None,
    'query': 'Zhou',
    'tweet_volume': 300026},
   {'name': 'Deji',
    'url': 'http://twitter.com/search?q=Deji',
    'promoted_content': None,
    'query': 'Deji',
    'tweet_volume': 25438},
   {'name': 'Sainz',
    'url': 'http://twitter.com/search?q=Sainz',
    'promoted_content': None,
    'query': 'Sainz',
    'tweet_volume': 211501},
   {'name': '#BritishGrandPrix',
    'url': 'http://twitter.com/search?q=%23BritishGrandPrix',
    'promoted_content': None,
    'query': '%23BritishGrandPrix',
    'tweet_volume': 25255},
   {'nam

In [14]:
# Look at the output as a DataFrame.

# Create a DataFrame.
london_trends_pd = pd.DataFrame(london[0]['trends'])

# View the DataFrame.
london_trends_pd

,name,url,promoted_content,query,tweet_volume
0,#BritishGP,http://twitter.com/search?q=%23BritishGP,None,%23BritishGP,553495.0
1,#SilverstoneGP,http://twitter.com/search?q=%23SilverstoneGP,None,%23SilverstoneGP,118576.0
2,Zhou,http://twitter.com/search?q=Zhou,None,Zhou,300026.0
3,Deji,http://twitter.com/search?q=Deji,None,Deji,25438.0
4,Sainz,http://twitter.com/search?q=Sainz,None,Sainz,211501.0
5,#BritishGrandPrix,http://twitter.com/search?q=%23BritishGrandPrix,None,%23BritishGrandPrix,25255.0
6,#Formula1,http://twitter.com/search?q=%23Formula1,None,%23Formula1,49681.0
7,#MondayMorning,http://twitter.com/search?q=%23MondayMorning,None,%23MondayMorning,NaN
8,Hamilton,http://twitter.com/search?q=Hamilton,None,Hamilton,221256.0
9,Happy 4th of July,http://twitter.com/search?q=%22Happy+4th+of+Ju...,None,%22Happy+4th+of+July%22,44742.0


In [15]:
# Narrow list down to 50,000 tweets.
london_trends_over50k_pd = london_trends_pd[london_trends_pd['tweet_volume'] > 50000]\
.sort_values('tweet_volume', ascending=False)

# View the output.
print(london_trends_over50k_pd.shape)
london_trends_over50k_pd

(23, 5)


,name,url,promoted_content,query,tweet_volume
0,#BritishGP,http://twitter.com/search?q=%23BritishGP,None,%23BritishGP,553495.0
2,Zhou,http://twitter.com/search?q=Zhou,None,Zhou,300026.0
20,Halo,http://twitter.com/search?q=Halo,None,Halo,241030.0
8,Hamilton,http://twitter.com/search?q=Hamilton,None,Hamilton,221256.0
17,Leclerc,http://twitter.com/search?q=Leclerc,None,Leclerc,214115.0
4,Sainz,http://twitter.com/search?q=Sainz,None,Sainz,211501.0
12,Ronaldo,http://twitter.com/search?q=Ronaldo,None,Ronaldo,210202.0
10,Lewis,http://twitter.com/search?q=Lewis,None,Lewis,182061.0
16,Perez,http://twitter.com/search?q=Perez,None,Perez,168459.0
21,Ferrari,http://twitter.com/search?q=Ferrari,None,Ferrari,160616.0


In [16]:
# Save output as a CSV file.
london_trends_over50k_pd.to_csv('london_trends_over50k.csv', index=False)

### compare cities

In [17]:
# Find New York.
our_city = 'New York'

# Create a variable.
new_york = [_ for _ in trends_worldwide if _['name'] == our_city]

# View the output.
new_york[0]['woeid']

2459115

In [18]:
# Find London.
our_city_2 = 'London'

# Create a variable.
london = [_ for _ in trends_worldwide if _['name'] == our_city_2]

# View the output.
london[0]['woeid']

44418

In [19]:
# Search for each city.
# Import JSON.
import json

# Search for New York.
new_york_trends = twitter_api.trends.place(_id=2459115)

# View JSON output.
print (json.dumps(new_york_trends, indent=4))

[
    {
        "trends": [
            {
                "name": "#July4th",
                "url": "http://twitter.com/search?q=%23July4th",
                "promoted_content": null,
                "query": "%23July4th",
                "tweet_volume": 14985
            },
            {
                "name": "#JaylandWalker",
                "url": "http://twitter.com/search?q=%23JaylandWalker",
                "promoted_content": null,
                "query": "%23JaylandWalker",
                "tweet_volume": 56709
            },
            {
                "name": "Breaking Bad",
                "url": "http://twitter.com/search?q=%22Breaking+Bad%22",
                "promoted_content": null,
                "query": "%22Breaking+Bad%22",
                "tweet_volume": 17501
            },
            {
                "name": "New Edition",
                "url": "http://twitter.com/search?q=%22New+Edition%22",
                "promoted_content": null,
                "que

In [20]:
# Search for London.
london_trends = twitter_api.trends.place(_id=44418)

# View JSON output.
print (json.dumps(london_trends, indent=4))

[
    {
        "trends": [
            {
                "name": "#BritishGP",
                "url": "http://twitter.com/search?q=%23BritishGP",
                "promoted_content": null,
                "query": "%23BritishGP",
                "tweet_volume": 553471
            },
            {
                "name": "#SilverstoneGP",
                "url": "http://twitter.com/search?q=%23SilverstoneGP",
                "promoted_content": null,
                "query": "%23SilverstoneGP",
                "tweet_volume": 118579
            },
            {
                "name": "Zhou",
                "url": "http://twitter.com/search?q=Zhou",
                "promoted_content": null,
                "query": "Zhou",
                "tweet_volume": 300039
            },
            {
                "name": "Deji",
                "url": "http://twitter.com/search?q=Deji",
                "promoted_content": null,
                "query": "Deji",
                "tweet_volume": 25

In [21]:
# Find common topics.
new_york_trends_list = [trend['name'] for trend in new_york_trends[0]['trends']]

# View output.
print(new_york_trends_list)

['#July4th', '#JaylandWalker', 'Breaking Bad', 'New Edition', 'The Wire', 'Happy 4th of July', 'The Sopranos', '#ESSENCEFEST', '#USFLChampionship', 'The Good Place', 'mad men', '#90DayFiance', 'The Office', 'Daily Quordle 161', 'Independence Day', 'Arrested Development', 'Wordle 380 X', 'Chan', 'Bojack Horseman', 'Ted Lasso', 'The West Wing', 'Better Call Saul', 'Game of Thrones', 'Bobby Brown', 'Johnny Gill', 'Fireworks', 'All in the Family', "Schitt's Creek", 'Ron Isley', 'Happy Fourth of July', 'The Simpsons', 'I Love Lucy', 'Six Feet Under', 'The Twilight Zone', 'Avatar the Last Airbender', 'PTSD', 'Gilda Radner', 'Pennsylvania', 'Abbott Elementary', 'Denmark', 'Isley Brothers', '30 Rock', 'Happy Birthday America', 'Cookus', 'Stallions', 'Modern Family', 'Infinity War', 'teedra moses', 'The Dark Knight', 'Wiggins']


In [22]:
# Find common topics.
london_trends_list = [trend['name'] for trend in london_trends[0]['trends']]

# View output.
print(london_trends_list)

['#BritishGP', '#SilverstoneGP', 'Zhou', 'Deji', '#BritishGrandPrix', 'Sainz', '#Formula1', '#MondayMotivation', 'Hamilton', 'Happy 4th of July', 'Lewis', 'Chyna', 'Ronaldo', 'Daily Quordle 161', 'Josh', 'Dami', 'Perez', 'Wordle 380 X', 'Leclerc', 'Verstappen', 'Halo', 'Ferrari', 'Kyrgios', 'Copenhagen', 'Coco', 'Sam Ryder', 'Indiyah', 'Samuel', 'George Russell', 'Coffey', 'Neil Oliver', 'Peter Brook', 'Martin Brundle', 'Mollie', 'Denmark', 'Tsitsipas', 'Mick', 'ronan keating', 'Tom Cruise', 'Gasly', 'nissany', 'Kohli', 'what a race', 'Djokovic', 'Albon', 'Theory', 'Izzy', 'cliff richard', 'Checo', 'Andrew']


In [23]:
# Find trends between cities.
new_york_trends_set = set(new_york_trends_list)
london_trends_set = set(london_trends_list)

# Set variable.
common_trends = new_york_trends_set.intersection(london_trends_set)

# View output.
print(common_trends)

{'Denmark', 'Wordle 380 X', 'Daily Quordle 161', 'Happy 4th of July'}


## Search for #Bitcoin

In [24]:
# Run a test with #Bitcoin.
bitcoin_tweets = twitter_api.search.tweets(q="#Bitcoin")

# View JSON output.
print(json.dumps(bitcoin_tweets, indent=4))

{
    "statuses": [
        {
            "created_at": "Mon Jul 04 05:53:31 +0000 2022",
            "id": 1543835376791478273,
            "id_str": "1543835376791478273",
            "text": "RT @cz_binance: Banks now use #bitcoin for reserves. Probably nothing.\n\n\u00bb Bank for International Settlements to allow banks to keep 1% of r\u2026",
            "truncated": false,
            "entities": {
                "hashtags": [
                    {
                        "text": "bitcoin",
                        "indices": [
                            30,
                            38
                        ]
                    }
                ],
                "symbols": [],
                "user_mentions": [
                    {
                        "screen_name": "cz_binance",
                        "name": "CZ \ud83d\udd36 Binance",
                        "id": 902926941413453824,
                        "id_str": "902926941413453824",
                        